In [1]:
import mne
import numpy as np
import pandas as pd
import seaborn as sea
import matplotlib.pyplot as plt
import os
from pathlib import Path
import random
import joblib 
import optuna

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from torch.utils.data import Dataset, DataLoader

from scipy.spatial import Delaunay
from scipy.interpolate import LinearNDInterpolator

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import GroupKFold

import warnings
warnings.filterwarnings('ignore')

In [2]:
SEED = 3126
EPOCHS = 35
EARLY_STOPPING_PATIENCE = 10

TIME_WINDOW = (.1,.5)
GRID_SIZE = 32
N_TIME_SLICES = 16
LEARNING_RATE = .001
WEIGHT_DECAY = .001

In [3]:
def set_seed(seed):
    random.seed(seed)               
    np.random.seed(seed)            
    torch.manual_seed(seed)         
    torch.cuda.manual_seed(seed)    
    torch.cuda.manual_seed_all(seed) 

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def worker_init_fn(worker_id):
    np.random.seed(SEED + worker_id)
    random.seed(SEED + worker_id)

set_seed(SEED)

In [4]:
class MEGDataset(Dataset):
    """
    Efficient MEG → 2D Topographic Image Dataset
    Uses precomputed Delaunay triangulation + LinearNDInterpolator for fast interpolation.
    """

    def __init__(self, X, y, epochs, scaler=None,
                 time_window=TIME_WINDOW, grid_size=GRID_SIZE, n_time_slices=N_TIME_SLICES):

        self.X = X
        self.y = y
        self.times = epochs.times
        self.info  = epochs.info
        self.grid_size = grid_size
        self.n_time_slices = n_time_slices

        ## Scale channels in data
        if scaler is None:
            ch_median = np.median(X, axis=(0,2))
            ch_iqr    = np.subtract(*np.percentile(X, [75, 25], axis=(0,2))) + 1e-6
            self.scaler   = (ch_median, ch_iqr)
        else: self.scaler = scaler
        ch_median, ch_iqr = self.scaler
        self.X = (self.X - ch_median[None,:,None]) / ch_iqr[None,:,None]

        # Find time window
        t0, t1 = time_window
        self.start = np.argmin(np.abs(self.times - t0))
        self.end   = np.argmin(np.abs(self.times - t1))
        self.slice_times = np.linspace(self.start, self.end-1, self.n_time_slices).astype(int)
        # Get sensor positions in 2D
        positions_3d = np.array([ch['loc'][:3] for ch in self.info['chs']])
        self.pos2d   = positions_3d[:, :2]

        # Get grid coordinates
        gx = np.linspace(self.pos2d[:,0].min(), self.pos2d[:,0].max(), grid_size)
        gy = np.linspace(self.pos2d[:,1].min(), self.pos2d[:,1].max(), grid_size)
        grid_x, grid_y = np.meshgrid(gx, gy)
        self.grid_points = np.c_[grid_x.ravel(), grid_y.ravel()].copy(order='C')
        # Gets triangulation
        self.triangulation = Delaunay(self.pos2d)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        trial = self.X[idx]     # shape: (channels, time)
        label = self.y[idx]
        topo_imgs = []

        # interpolation of each time slice
        for t in self.slice_times:
            channels = trial[:, t]  # (C,)
            fill_val = np.median(channels)
            interpolator = LinearNDInterpolator(self.triangulation, channels, fill_value=fill_val)
            topo_img = interpolator(self.grid_points).reshape(self.grid_size, self.grid_size)
            topo_img = (topo_img - topo_img.mean(axis=0))/(topo_img.std(axis=0) + 1e-6)
            topo_imgs.append(np.nan_to_num(topo_img))

        topo_imgs = np.stack(topo_imgs, axis=0)  # (n_time_slices, grid_size, grid_size)
        return torch.tensor(topo_imgs, dtype=torch.float32), torch.tensor(label, dtype=torch.long)

In [5]:
# X, y, groups, first_epochs, _ = load_all_subjects_data("/kaggle/input/the-imagine-decoding-challenge/train/train", False, "localizer")
# data = MEGDataset(X,y,first_epochs)
# sample,_ = data[0]

# for k in range(N_TIME_SLICES):
#     sea.heatmap(sample[k])
#     # sea.heatmap((sample[k]-sample[k].mean(axis=0))/(sample[k].std(axis=0) + 1e-6))
#     plt.show()

In [6]:
class SqueezeExcitation(nn.Module):
    def __init__(self, channels, reduction=16):
        super(SqueezeExcitation, self).__init__()
        hidden = max(channels//reduction, 1)
        self.squeeze    = nn.AdaptiveAvgPool2d(1)
        self.excitation = nn.Sequential(
            nn.Linear(channels, hidden, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(hidden, channels, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        # Squeeze: (B,C,1,1) → (B,C)
        s = self.squeeze(x).view(b, c)
        # Excitation: (B,C) → (B, C, 1, 1)
        w = self.excitation(s).view(b, c, 1, 1)
        return x*w

In [7]:
class Model(nn.Module):
    def __init__(self, in_channels=N_TIME_SLICES, hidden_channels=64, conv_dropout=.2, classifier_dropout=.2, reduction=16,
                 conv_activation="relu", n_classes=10):
        super(Model, self).__init__()
        
        ACTIVATIONS = {"relu": nn.ReLU, "silu": nn.SiLU, "gelu": nn.GELU}

        def conv_block(in_channels, out_channels):
            return nn.Sequential(
                nn.Conv2d(in_channels, out_channels, 3, padding=1, bias=False),
                nn.BatchNorm2d(out_channels),
                ACTIVATIONS[conv_activation](),
                SqueezeExcitation(out_channels, reduction),
                nn.Dropout(conv_dropout))
                
        def linear_block(in_dim, out_dim):
            return nn.Sequential(
                nn.Linear(in_dim, out_dim, bias=True),
                nn.ReLU(),
                nn.Dropout(classifier_dropout))
            
        self.conv = nn.Sequential(
            conv_block(in_channels, hidden_channels),
            conv_block(hidden_channels, 2*hidden_channels),
            nn.MaxPool2d(2),
            conv_block(2*hidden_channels, 2*hidden_channels),
            conv_block(2*hidden_channels, 4*hidden_channels),
            nn.MaxPool2d(2)
        )

        self.global_pool = nn.AdaptiveAvgPool2d(1)
            
        self.classifier = nn.Sequential(
            linear_block(4*hidden_channels, 256),
            linear_block(256, 64),
            nn.Linear(64, n_classes)
        )
        
    def forward(self, x):
        o = self.conv(x)
        o = self.global_pool(o)  # shape → (batch, channels, 1)
        o = o.view(o.size(0), -1)   # shape → (batch, channels)
        return self.classifier(o)

In [8]:
def init_model_weights(model):
    for module in model.modules():
        ## Linear and Convolution 
        if isinstance(module, (nn.Linear, nn.Conv2d)):
            nn.init.kaiming_uniform_(module.weight, mode="fan_in", nonlinearity="relu")
            if module.bias is not None:
                nn.init.zeros_(module.bias)
        ## Batchnorm 
        if isinstance(module, (nn.BatchNorm2d, nn.BatchNorm1d)):
            nn.init.ones_(module.weight)
            nn.init.zeros_(module.bias)

In [9]:
def train_evaluate(model, train_loader, val_loader, save_path, lr, weight_decay, 
                   epochs=EPOCHS, early_stopping_patience=EARLY_STOPPING_PATIENCE, device="cuda"):
    model.to(device)
    optimizer    = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    loss_fn      = nn.CrossEntropyLoss()
    lr_scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=len(train_loader))
    history = {"train_losses":[], "val_losses":[], "train_accs":[], "val_accs":[]}

    min_change          = .002
    best_val_acc        = 0.0
    epochs_not_improved = 0
    
    for epoch in range(epochs):
        # -------------------- Model training -------------------- 
        model.train()
        train_loss = 0.0
        train_acc  = 0.0
        total      = 0.0
        for X, y in train_loader:
            X, y   = X.to(device), y.to(device)
            logits = model(X)
            loss   = loss_fn(logits, y)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            lr_scheduler.step()
            
            ## Accumulates loss and accuracy
            train_loss += loss.item()
            preds       = logits.argmax(dim=1)
            train_acc  += (preds==y).sum().item()
            total      += y.size(0)
        train_loss /= len(train_loader)
        train_acc  /= total
        
        # -------------------- Model Validation -------------------- 
        model.eval()
        val_loss = 0.0
        val_acc  = 0.0
        total    = 0.0
        with torch.inference_mode():
            for X, y in val_loader:
                X, y  = X.to(device), y.to(device)

                ## Compute loss
                logits    = model(X)
                loss      = loss_fn(logits, y)
                val_loss += loss.item()
                ## Tracks validation accuracy
                preds    = logits.argmax(dim=1)
                val_acc += (preds==y).sum().item()
                total   += y.size(0)
        val_loss /= len(val_loader)
        val_acc  /= total

        ## -------------------- Logging -------------------- 
        history["train_losses"].append(train_loss)
        history["val_losses"].append(val_loss)
        history["train_accs"].append(train_acc)
        history["val_accs"].append(val_acc)
        
        print(f"Epoch {epoch+1}/{epochs} | ",
              f"Train Loss: {train_loss:.4f} | ",
              f"Val Loss: {val_loss:.4f} | ",
              f"Train Acc: {train_acc:.3f} | ",
              f"Val Acc: {val_acc:.3f}")

        # ----------------- Early Stopping -----------------
        if val_acc - best_val_acc >= min_change:
            best_val_acc        = val_acc
            epochs_not_improved = 0
            torch.save(model.state_dict(), save_path)
        else:
            epochs_not_improved += 1
            print(f"Early Stopping Counter: {epochs_not_improved}/{early_stopping_patience}")
        if epochs_not_improved >= early_stopping_patience:
            print(f"Early stopping triggered at epoch {epoch+1}")
            break
    return history


def predict(model, loader, device="cuda"):
    model.eval()
    all_preds = []
    with torch.inference_mode():
        for X,_ in loader:
            X = X.to(device)
            all_preds.append(model(X).argmax(dim=1))
    return torch.cat(all_preds).cpu().numpy()

In [10]:
def load_subject_data(data_path, subject_id, need_label_map=True, data_type='localizer'):
    """
    Load data for a single subject.
    Returns:
        X: ndarray (M_trials, C_channels, T_timepoints)
        y: labels  (M_trials,)
        epochs: MNE epochs object
        label_map: dict mapping event names to codes
    """
    file_path = Path(data_path) / subject_id / f"{subject_id}_{data_type}-epo.fif"
    epochs    = mne.read_epochs(file_path, preload=True, verbose=False)
    X         = epochs.get_data()
    y         = epochs.events[:,2]-1  # ranges from [1, 10], subtracts 1 to become [0,9]
    label_map = None
    if need_label_map: 
        label_map = {key:value-1 for key,value in epochs.event_id.items()} # shift values down to be in range [0,9]
    return X, y, epochs, label_map



def load_all_subjects_data(data_path, need_label_map=True, data_type='localizer'):
    """
    Load data for all subjects.
    Returns:
        X: ndarray (M_trials * num_subjects, C_channels, T_timepoints)
        y: labels  (M_trials * num_subjects,)
        groups: ndarray
        label_map: dict
    """
    subject_ids = os.listdir(data_path)
    all_X, all_y, all_groups, first_epochs = [], [], [], None
    
    for idx, subject_id in enumerate(subject_ids):
        X, y, epochs, label_map = load_subject_data(data_path, subject_id, need_label_map, data_type)
        if first_epochs is None: first_epochs=epochs
        all_X.append(X)
        all_y.append(y)
        all_groups.append(np.full(len(y), idx))
    
    X = np.concatenate(all_X, axis=0) 
    y = np.concatenate(all_y, axis=0) 
    groups = np.concatenate(all_groups, axis=0)
    return X, y, groups, first_epochs, label_map

In [11]:
def cross_validate(data_path, n_splits=5, 
                   dataset_params=None, model_params=None, training_params=None, device='cuda'):
    # LOAD ALL SUBJECTS
    X, y, groups, first_epochs, _ = load_all_subjects_data(data_path, False, "localizer")
    print(f"Total trials: {len(y)} | Subjects: {len(os.listdir(data_path))} | X dimension: {X.shape}")
    
    # --- GroupKFold CV ---
    gkf = GroupKFold(n_splits=n_splits)
    fold_scores = []
    
    for fold_idx, (train_idx, val_idx) in enumerate(gkf.split(X, y, groups)):
        print(f"\nFold {fold_idx+1}/{n_splits}")

        ## Create train and validation datasets and loaders
        X_train, y_train = X[train_idx], y[train_idx]
        X_val,   y_val   = X[val_idx],   y[val_idx]
        train_data   = MEGDataset(X_train, y_train, first_epochs, **dataset_params)
        val_data     = MEGDataset(X_val, y_val, first_epochs, train_data.scaler, **dataset_params)
        train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=4, worker_init_fn=worker_init_fn)
        val_loader   = DataLoader(val_data,   batch_size=32, shuffle=False, num_workers=4, worker_init_fn=worker_init_fn)

        ## Trains model
        save_path = f"/kaggle/working/best_model_fold_{fold_idx+1}.pth"
        model     = Model(**model_params).to(device)
        init_model_weights(model)
        history   = train_evaluate(model, train_loader, val_loader, save_path, **training_params, device=device)
        
        # Evaluates model
        model = Model(**model_params).to(device)
        model.load_state_dict(torch.load(save_path))
        val_preds = predict(model, val_loader, device=device)
        val_acc   = (val_preds==y_val).mean()
        fold_scores.append(val_acc)
        print(f"Fold {fold_idx+1} Validation Accuracy: {val_acc:.4f}\n\n")
    
    mean_acc, std_acc = np.mean(fold_scores), np.std(fold_scores)
    print(f"\nMean Accuracy: {mean_acc:.4f} ± {std_acc:.4f}")
    return fold_scores, mean_acc, std_acc

In [12]:
# dataset_params  = {'time_window':TIME_WINDOW, 'grid_size':GRID_SIZE, 'n_time_slices':N_TIME_SLICES}
# model_params    = {'in_channels':N_TIME_SLICES, 'n_classes':10}
# training_params = {'epochs':EPOCHS, 'lr':LEARNING_RATE, "weight_decay":WEIGHT_DECAY}

# # Cross-validation
# cv_scores, mean_acc, std_acc = cross_validate(
#     '/kaggle/input/the-imagine-decoding-challenge/train/train', 
#     n_splits=5, 
#     dataset_params=dataset_params,
#     model_params=model_params,
#     training_params=training_params,
#     device='cuda'
# )

# # Train on all & predict test
# # submission = train_and_predict(
# #     '/kaggle/input/the-imagine-decoding-challenge/train/train', 
# #     '/kaggle/input/the-imagine-decoding-challenge/test/test', 
# #     dataset_params=dataset_params,
# #     model_params=model_params,
# #     training_params=training_params,
# #     output_file='submission.csv',
# #     device='cpu'
# # )

In [13]:
def objective(trial):
    t0 = trial.suggest_float("time_window_t0", 0.0, 0.25)
    t1 = trial.suggest_float("time_window_t1", 0.5, 0.85)
    
    dataset_params  = {
        'time_window'  : (t0, t1), 
        'grid_size'    : trial.suggest_categorical("grid_size", [32, 48, 56, 64, 72]), 
        'n_time_slices': trial.suggest_categorical("n_time_slices", [8, 12, 16, 32, 48])
    }

    model_params = {
        'in_channels'       :dataset_params["n_time_slices"], 
        'hidden_channels'   :trial.suggest_int("hidden_channels", dataset_params["n_time_slices"], 4*dataset_params["n_time_slices"]),
        'conv_dropout'      :trial.suggest_float("conv_dropout", 0.1, 0.5),
        'classifier_dropout':trial.suggest_float("classifier_dropout", 0.1, 0.5),
        'conv_activation'   :trial.suggest_categorical("conv_activation", ["relu", "silu", "gelu"]),
    }
    model_params["reduction"] = trial.suggest_int("reduction", 8, model_params["hidden_channels"])
    
    training_params = {
        'lr':trial.suggest_float("lr", .0001, .01, log=True), 
        "weight_decay":trial.suggest_float("weight_decay", .0001, .01, log=True)
    }
    
    cv_scores, mean_acc, std_acc = cross_validate(
        '/kaggle/input/the-imagine-decoding-challenge/train/train', 
        n_splits=3, 
        dataset_params=dataset_params,
        model_params=model_params,
        training_params=training_params,
        device='cuda'
    )
    return mean_acc,std_acc

study = optuna.create_study(directions=["maximize", "minimize"])  
study.optimize(objective, n_trials=25)

[I 2025-11-27 04:13:30,409] A new study created in memory with name: no-name-82fb7ddb-cffd-4ee4-bd16-8f2c0e414e85


Total trials: 7200 | Subjects: 15 | X dimension: (7200, 309, 121)

Fold 1/3
Epoch 1/35 |  Train Loss: 2.3165 |  Val Loss: 2.3029 |  Train Acc: 0.102 |  Val Acc: 0.100
Epoch 2/35 |  Train Loss: 2.3055 |  Val Loss: 2.3027 |  Train Acc: 0.097 |  Val Acc: 0.098
Early Stopping Counter: 1/10
Epoch 3/35 |  Train Loss: 2.3039 |  Val Loss: 2.3027 |  Train Acc: 0.091 |  Val Acc: 0.098
Early Stopping Counter: 2/10
Epoch 4/35 |  Train Loss: 2.3043 |  Val Loss: 2.3027 |  Train Acc: 0.096 |  Val Acc: 0.100
Early Stopping Counter: 3/10
Epoch 5/35 |  Train Loss: 2.3032 |  Val Loss: 2.3026 |  Train Acc: 0.095 |  Val Acc: 0.100
Early Stopping Counter: 4/10
Epoch 6/35 |  Train Loss: 2.3033 |  Val Loss: 2.3026 |  Train Acc: 0.098 |  Val Acc: 0.100
Early Stopping Counter: 5/10
Epoch 7/35 |  Train Loss: 2.3032 |  Val Loss: 2.3026 |  Train Acc: 0.094 |  Val Acc: 0.100
Early Stopping Counter: 6/10
Epoch 8/35 |  Train Loss: 2.3038 |  Val Loss: 2.3027 |  Train Acc: 0.090 |  Val Acc: 0.100
Early Stopping Counter

[I 2025-11-27 04:44:56,763] Trial 0 finished with values: [0.10083333333333333, 0.0014829275350043505] and parameters: {'time_window_t0': 0.24330342354389584, 'time_window_t1': 0.7896634999033203, 'grid_size': 64, 'n_time_slices': 32, 'hidden_channels': 43, 'conv_dropout': 0.28912349323458486, 'classifier_dropout': 0.23163117801630728, 'conv_activation': 'relu', 'reduction': 27, 'lr': 0.00398864357997797, 'weight_decay': 0.0004980261139185028}.


Fold 3 Validation Accuracy: 0.1029



Mean Accuracy: 0.1008 ± 0.0015
Total trials: 7200 | Subjects: 15 | X dimension: (7200, 309, 121)

Fold 1/3
Epoch 1/35 |  Train Loss: 2.3286 |  Val Loss: 2.3057 |  Train Acc: 0.097 |  Val Acc: 0.097
Epoch 2/35 |  Train Loss: 2.3069 |  Val Loss: 2.3040 |  Train Acc: 0.092 |  Val Acc: 0.102
Epoch 3/35 |  Train Loss: 2.3059 |  Val Loss: 2.3048 |  Train Acc: 0.102 |  Val Acc: 0.100
Early Stopping Counter: 1/10
Epoch 4/35 |  Train Loss: 2.3060 |  Val Loss: 2.3034 |  Train Acc: 0.104 |  Val Acc: 0.097
Early Stopping Counter: 2/10
Epoch 5/35 |  Train Loss: 2.3047 |  Val Loss: 2.3026 |  Train Acc: 0.104 |  Val Acc: 0.095
Early Stopping Counter: 3/10
Epoch 6/35 |  Train Loss: 2.3048 |  Val Loss: 2.3027 |  Train Acc: 0.102 |  Val Acc: 0.101
Early Stopping Counter: 4/10
Epoch 7/35 |  Train Loss: 2.3045 |  Val Loss: 2.3029 |  Train Acc: 0.099 |  Val Acc: 0.098
Early Stopping Counter: 5/10
Epoch 8/35 |  Train Loss: 2.3047 |  Val Loss: 2.3028 |  Train Acc: 0.104 

[I 2025-11-27 05:00:53,733] Trial 1 finished with values: [0.10569444444444444, 0.002315740555629594] and parameters: {'time_window_t0': 0.1010458245266995, 'time_window_t1': 0.7283106018878328, 'grid_size': 48, 'n_time_slices': 16, 'hidden_channels': 46, 'conv_dropout': 0.481265143861929, 'classifier_dropout': 0.3808899402215068, 'conv_activation': 'silu', 'reduction': 20, 'lr': 0.0008807592673789643, 'weight_decay': 0.0001417054818043367}.


...

Fold 3 Validation Accuracy: 0.1071



Mean Accuracy: 0.1046 ± 0.0020
Total trials: 7200 | Subjects: 15 | X dimension: (7200, 309, 121)

Fold 1/3
Epoch 1/35 |  Train Loss: 2.3466 |  Val Loss: 2.3031 |  Train Acc: 0.088 |  Val Acc: 0.100
Epoch 2/35 |  Train Loss: 2.3045 |  Val Loss: 2.3027 |  Train Acc: 0.100 |  Val Acc: 0.107
Epoch 3/35 |  Train Loss: 2.3080 |  Val Loss: 2.3031 |  Train Acc: 0.094 |  Val Acc: 0.100
Early Stopping Counter: 1/10
Epoch 4/35 |  Train Loss: 2.3039 |  Val Loss: 2.3028 |  Train Acc: 0.099 |  Val Acc: 0.100
Early Stopping Counter: 2/10
Epoch 5/35 |  Train Loss: 2.3037 |  Val Loss: 2.3030 |  Train Acc: 0.091 |  Val Acc: 0.094
Early Stopping Counter: 3/10
Epoch 6/35 |  Train Loss: 2.3033 |  Val Loss: 2.3031 |  Train Acc: 0.098 |  Val Acc: 0.102
Early Stopping Counter: 4/10
Epoch 7/35 |  Train Loss: 2.3039 |  Val Loss: 2.3029 |  Train Acc: 0.092 |  Val Acc: 0.100
Early Stopping Counter: 5/10
Epoch 8/35 |  Train Loss: 2.3040 |  Val Loss: 2.3035 |  Train Acc: 0.095 

[I 2025-11-27 12:26:15,065] Trial 23 finished with values: [0.10222222222222221, 0.0031426968052735453] and parameters: {'time_window_t0': 0.1693799789096838, 'time_window_t1': 0.6550504908985426, 'grid_size': 48, 'n_time_slices': 48, 'hidden_channels': 191, 'conv_dropout': 0.4014969901998089, 'classifier_dropout': 0.4091713816729484, 'conv_activation': 'silu', 'reduction': 32, 'lr': 0.007206839686804187, 'weight_decay': 0.0010122098336901799}.


Fold 3 Validation Accuracy: 0.1000



Mean Accuracy: 0.1022 ± 0.0031
Total trials: 7200 | Subjects: 15 | X dimension: (7200, 309, 121)

Fold 1/3
Epoch 1/35 |  Train Loss: 2.4031 |  Val Loss: 2.3071 |  Train Acc: 0.106 |  Val Acc: 0.095
Epoch 2/35 |  Train Loss: 2.3224 |  Val Loss: 2.3043 |  Train Acc: 0.090 |  Val Acc: 0.097
Early Stopping Counter: 1/10
Epoch 3/35 |  Train Loss: 2.3124 |  Val Loss: 2.3041 |  Train Acc: 0.094 |  Val Acc: 0.098
Epoch 4/35 |  Train Loss: 2.3071 |  Val Loss: 2.3036 |  Train Acc: 0.099 |  Val Acc: 0.097
Early Stopping Counter: 1/10
Epoch 5/35 |  Train Loss: 2.3055 |  Val Loss: 2.3034 |  Train Acc: 0.100 |  Val Acc: 0.098
Early Stopping Counter: 2/10
Epoch 6/35 |  Train Loss: 2.3053 |  Val Loss: 2.3031 |  Train Acc: 0.106 |  Val Acc: 0.098
Early Stopping Counter: 3/10
Epoch 7/35 |  Train Loss: 2.3044 |  Val Loss: 2.3029 |  Train Acc: 0.111 |  Val Acc: 0.101
Epoch 8/35 |  Train Loss: 2.3055 |  Val Loss: 2.3031 |  Train Acc: 0.099 |  Val Acc: 0.097
Early Stopp

[I 2025-11-27 12:44:05,471] Trial 24 finished with values: [0.10347222222222223, 0.002576005137637695] and parameters: {'time_window_t0': 0.24158525667199635, 'time_window_t1': 0.5135942891293761, 'grid_size': 64, 'n_time_slices': 12, 'hidden_channels': 41, 'conv_dropout': 0.42982252761103457, 'classifier_dropout': 0.36411917927140636, 'conv_activation': 'relu', 'reduction': 27, 'lr': 0.00022450276046966547, 'weight_decay': 0.000579745264309815}.


Fold 3 Validation Accuracy: 0.1021



Mean Accuracy: 0.1035 ± 0.0026
